# Introduction

In this notebook, I'll explore the TPS Jan 2022 dataset in order to forecast the sales of certain products from two stores. Several feature engineering techniques are reviewed (especially the interaction betweeen features performed in this [notebook](https://www.kaggle.com/lucamassaron/kaggle-merchandise-eda-with-baseline-linear-model/notebook) by [Luca Massaron](https://www.kaggle.com/lucamassaron)).

# Libraries and data

In [ ]:
# Libraries needed

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Importing the train and test datasets
train = pd.read_csv("../input/tabular-playground-series-jan-2022/train.csv", parse_dates=['date'])
test = pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv", parse_dates=['date'])

# Obtaining the full dataset with a label train/test
full_df = pd.concat([train.assign(dataset='train'), test.assign(dataset='test')])

# Notice we get NaN values for the unknown sales of the test dataset
full_df

Let's quickly confirm what countries, store and products belong to this dataset.

In [ ]:
print('Countries:', full_df['country'].unique().tolist())
print('Stores:', full_df['store'].unique().tolist())
print('Products:', full_df['product'].unique().tolist())

In order to be sure which years we're working on, let's create a 'year' column from the date timestamps.

In [ ]:
full_df['year'] = full_df['date'].apply(lambda date: date.year) 
full_df['year'].unique()

It should be noted that the dates are already sorted and include all days from 2015 to 2019. To confirm this, we can generate a timestamp for every day in these years with pandas, and then compare it to our date columns.

In [ ]:
# Array of all days from the first day of 2015 to the last day of 2019
days_range = pd.date_range(start='2015-01-01', end='2019-12-31')

# Is this equal to the (unique) dates from our date column?
(days_range != full_df['date'].unique()).sum()

# Amount of falses is zero:

We're also provided with the GDP of the three countries during these years. In previous versions, I used the GDP values provided by [Carl McBride](https://www.kaggle.com/carlmcbrideellis/gdp-20152019-finland-norway-and-sweden). 

In this version, however, I use the GDP per capita dataset by [Samuel Cortinhas](https://www.kaggle.com/samuelcortinhas/gdp-per-capita-finland-norway-sweden-201519), as it has improved the score by a bit.

In [ ]:
gdp = pd.read_csv('../input/gdp-per-capita-finland-norway-sweden-201519/GDP_per_capita_2015_to_2019_Finland_Norway_Sweden.csv')
gdp.columns = ['year', 'GDP_Finland', 'GDP_Norway', 'GDP_Sweden']
gdp

# Exploratory Data Analysis

First, we want to make sure which years correspond to the train dataset, and which years to the test dataset.

In [ ]:
plt.figure(figsize=(8,3))
sns.countplot(data=full_df, x='year', hue='dataset', palette='Set2')
plt.legend(title='dataset', bbox_to_anchor=(1.05, 1));

From the above result, we see that it would be a good idea to train our model on the years 2015, 2016 and 2017 and then evaluate it on 2018 as a hold-out test set.

Let's first see what the sales look like between the three available products.

In [ ]:
plt.figure(figsize=(15,6))
sns.lineplot(data=full_df[full_df['year']<2019], x='date', y='num_sold', hue='product');

From the above graph, we can clearly infer that there's a certain sales seasonality, and that we can order the amounf of sales as Hat > Mug > Sticker.

Also, we should check if a store sells more than the other two.

In [ ]:
plt.figure(figsize=(15,6))
sns.lineplot(data=full_df[full_df['year']<2019], x='date', y='num_sold', hue='store', palette='hls');

So it seems that KaggleRama sells more products than KaggleMart.

We should also check how the sales differ from one country to another.

In [ ]:
plt.figure(figsize=(15,6))
sns.lineplot(data=full_df[full_df['year']<2019], x='date', y='num_sold', hue='country', palette='Set1');

We see that it's pretty much Norway > Sweden > Finland, regarding the amount of sales by country. 

# Feature Engineering

Since the seasonily is important, we should get as much information as possible from the date column, such as year (which we already did), month, week, day of the month and day of the week.

In [ ]:
full_df['month'] = full_df['date'].dt.month
full_df['week'] = full_df['date'].dt.isocalendar().week
full_df['week'][full_df['week']>52] = 52
full_df['day'] = full_df['date'].dt.day
full_df['dayofweek'] = full_df['date'].dt.dayofweek

full_df.head()

Next, we want to create interactions between features. To understand how this works, first pick up a couple of rows as an example:

In [ ]:
rows_example = full_df.iloc[:2]
rows_example

Say we want to create interactions between three features: country, product and month. For every row, we will join their (string) values along the mentioned columns (features).

In [ ]:
# Features we want to interact between each other
features_example = ['country', 'product', 'month']

# For the two selected rows, join their respective values
interactions = rows_example[features_example].apply(lambda row: '_'.join(row.astype(str)), axis=1)
interactions

In [ ]:
# One-hot encode them
pd.get_dummies(interactions)

If we select the rows' features we're interested in, and concat them to the above frame, we get something like this:

In [ ]:
pd.concat((rows_example[features_example], pd.get_dummies(interactions)), axis=1)

In other words, since the first row satisfies having Finland as the country, Kaggle Mug as the product and month equal to 1 (January), it gets a 1 (one) in the 'Finland_Kaggle Mug_1' column, but a 0 in the Kaggle Hat column, whereas the second row gets a 1 there.

Now, to the real work, we will create four interactions and perform the process on the full dataframe as follows. 
(Also, keep in mind that too many interactions can result in overfit when training the model, as I found out when I was testing this, as well as an insanely long time training the model.)

In [ ]:
desired_interactions = [
        ['country', 'product', 'month'],
        ['country', 'product', 'week'],
        ['country', 'store', 'week'],
        ['country', 'product', 'month', 'day'],
        ]

for interaction in desired_interactions:
    
    interaction_features = full_df[interaction].apply(lambda row: '_'.join(row.astype(str)), axis=1)
    interaction_features = pd.get_dummies(interaction_features)
    
    full_df = pd.concat((full_df, interaction_features), axis=1)

It should be noted that it may appear that we get columns with the same name, because the month and week values can overlap as in the following example (first month (1) plus first week (1)). But this isn't directly a problem so we leave it as is. 

In [ ]:
full_df[['country', 'product', 'month', 'week', 'Finland_Kaggle Hat_1']].head()

We will now add the GDP values of each country. Remember we had the GDP dataset:

In [ ]:
gdp

We will create three new columns 'GDP_Finland', 'GDP_Norway', 'GDP_Sweden" such that their values will be either zero or the corresponding GDP value if the country and the year match with those of the row.

In [ ]:
# Function to get the GDP value from the dataset
def get_gdp(country_from_row, year, country):
    
    if country_from_row == country:
        var_name = 'GDP_' + country
        return gdp[gdp['year']==year][var_name].values[0]
    
    else: 
        return 0

In [ ]:
# Example 
# If the row's country matches the country we choose, return the GDP 
print( get_gdp('Sweden', 2018, 'Sweden') )

# Else return 0
print( get_gdp('Finland', 2019, 'Sweden') )

In [ ]:
# Creating the columns with the corresponding GDP values

full_df['GDP_Finland'] = np.vectorize(get_gdp, otypes=['float'])(
    full_df['country'], full_df['year'], 'Finland' )

full_df['GDP_Norway'] = np.vectorize(get_gdp, otypes=['float'])(
    full_df['country'], full_df['year'], 'Norway' )

full_df['GDP_Sweden'] = np.vectorize(
    get_gdp, otypes=['float'])(
    full_df['country'], full_df['year'], 'Sweden' )

In [ ]:
# Random example

full_df[['country', 'year', 'GDP_Finland', 'GDP_Norway', 'GDP_Sweden']].iloc[[2000, 15000, 25000]]

We also drop the 'date' and 'dataset' columns as we won't use them anymore.

In [ ]:
full_df.drop(['date', 'dataset'], axis=1, inplace=True)
full_df.head()

Now we will perform one-hot encoding for the categorical features, namely: country, store, product, month, week, day and dayofweek. Note that although these last features are stored as integers, they should be treated as categorical since they don't actually follow a certain hierarchical order. Thus, we will convert them to strings and then get the dummies of these categorical features from the entire dataframe.

In [ ]:
# List of categorical features
cat_features = ['country', 'store', 'product', 'month', 'week', 'day', 'dayofweek']

# Convert all of them to strings just in case
full_df[cat_features] = full_df[cat_features].astype(str)

# Getting dummies for the selected columns
full_df = pd.get_dummies(full_df)

full_df.head()

Finally, we can get our dataset ready for our model to be trained. Remember that we first train the model on the years 2015, 2016 and 2017, so that we can evaluate on 2018.

In [ ]:
X_train = full_df[full_df['year']<2018].drop(['row_id', 'num_sold'], axis=1)
y_train = full_df[full_df['year']<2018]['num_sold']

X_test = full_df[full_df['year']==2018].drop(['row_id', 'num_sold'], axis=1)
y_test = full_df[full_df['year']==2018]['num_sold']

# Model and Evaluation

To evaluate the model, we consider the SMAPE measure.

In [ ]:
def SMAPE(y_true, y_pred):
    # Reference  https://www.kaggle.com/cpmpml/smape-weirdness
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)

We will construct a linear multiplicative model. Thus, we will import the LinearRegression model from scikit learn, and we will also transform the label 'y' via the natural logarithm. 

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Creating an instance of a Linear Regression
model = LinearRegression()

# Training the model
model.fit(X_train, np.log(y_train))

First, let's make sure that the model actually got close to the very y_train.

In [ ]:
# Computing the predicted values from X_train
train_predictions = model.predict(X_train)

# Remember to inverse-transform via the exp function
train_predictions = np.exp(train_predictions)

# Calculating the SMAPE
SMAPE(y_train, train_predictions)

We now evaluate on 2018.

In [ ]:
test_predictions = model.predict(X_test)
test_predictions = np.exp(test_predictions)

SMAPE(y_test, test_predictions)

This isn't too bad, so we retrain the model including 2018 and make our predictions on 2019 for the competition.

In [ ]:
X_full_train = full_df[full_df['year']<2019].drop(['row_id', 'num_sold'], axis=1)
y_full_train = full_df[full_df['year']<2019]['num_sold']

In [ ]:
model = LinearRegression()
model.fit(X_full_train, np.log(y_full_train))

In [ ]:
# Grabbing the rows from 2019
X_competition = full_df[full_df['year']==2019].drop(['row_id', 'num_sold'], axis=1)

# Calculating our predictions
final_predictions = np.exp(model.predict(X_competition))

# Formatting for the submission, including the row_id
X_competition['num_sold'] = final_predictions
X_competition['row_id'] = full_df[full_df['year']==2019]['row_id']
X_competition = X_competition[['row_id', 'num_sold']]

X_competition

At last, the submission!

In [ ]:
X_competition.to_csv('submission.csv', index=False)

That's all. Thank you for reading.